In [74]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import statsmodels.api as sm
plt.style.use('fivethirtyeight')
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_predict
from statsmodels.tsa.stattools import adfuller, acf, pacf
from sklearn.metrics import mean_squared_error

df_basinc = pd.read_csv('/Users/ardahanaytan/Desktop/STAJ/Hava Durumu Tahmin Projesi/data/weather_datas/basinc.csv')
df_bulut = pd.read_csv('/Users/ardahanaytan/Desktop/STAJ/Hava Durumu Tahmin Projesi/data/weather_datas/bulut.csv')
df_nem = pd.read_csv('/Users/ardahanaytan/Desktop/STAJ/Hava Durumu Tahmin Projesi/data/weather_datas/nem.csv')
df_ruzgar = pd.read_csv('/Users/ardahanaytan/Desktop/STAJ/Hava Durumu Tahmin Projesi/data/weather_datas/ruzgar.csv')
df_sicaklik = pd.read_csv('/Users/ardahanaytan/Desktop/STAJ/Hava Durumu Tahmin Projesi/data/weather_datas/sicaklik.csv')
df_yagmur = pd.read_csv('/Users/ardahanaytan/Desktop/STAJ/Hava Durumu Tahmin Projesi/data/weather_datas/yagmur.csv')
df_kar = pd.read_csv('/Users/ardahanaytan/Desktop/STAJ/Hava Durumu Tahmin Projesi/data/weather_datas/kar.csv')

tables = [df_nem, df_ruzgar, df_sicaklik, df_yagmur, df_kar]
df = pd.merge(df_basinc, df_bulut, on=['Zaman'])
for i in tables:
    df = pd.merge(df, i, on=['Zaman'])

df = df.dropna()
df['Yil'] = df['Zaman'].str[:4].astype(int)
df['Ay'] = df['Zaman'].str[4:6].astype(int)
df['Gun'] = df['Zaman'].str[6:8].astype(int)
df['Saat'] = df['Zaman'].str[9:].str[:2].astype(int)
#df['Zaman'] = df[['Yil', 'Ay', 'Gun']].astype(str).agg('-'.join, axis=1)
df['datetime'] = pd.to_datetime(df[['Yil', 'Ay', 'Gun']].astype(str).agg('-'.join, axis=1) + ' ' + 
                                df['Saat'].astype(str), format='%Y-%m-%d %H')
df.set_index('datetime', inplace=True)
df.drop(columns=['Zaman', 'Yil', 'Ay', 'Gun', 'Saat'], inplace=True)

In [75]:
df_y = pd.DataFrame(df['Basinc'])
df_max = df_y.resample('D').max()
df_max.rename(columns = {'Basinc': 'max'}, inplace=True)
df_min = df_y.resample('D').min()
df_min.rename(columns = {'Basinc': 'min'}, inplace=True)
df_top = df_y.resample('D').sum()
df_top.rename(columns = {'Basinc': 'sum'}, inplace=True)
df_ort = df_y.resample('D').mean()
df_ort.rename(columns = {'Basinc': 'mean'}, inplace=True)
df_ya = pd.concat([df_top, df_ort, df_max, df_min], axis=1)
df_ya['target'] = df_ya.shift(-1)['max']
df_ya = df_ya.iloc[:-1,:].copy()

In [76]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

reg = Ridge(alpha=.1)
predictors = ['sum', 'mean', 'max', 'min']

def create_predictions(predictors, core_weather, reg):
    train = df_ya.loc[:'2021-12-31']
    test = df_ya.loc['2022-01-01':]
    reg.fit(train[predictors], train['target'])
    predictions = reg.predict(test[predictors])
    error = mean_absolute_error(test['target'], predictions)
    combined = pd.concat([test['target'], pd.Series(predictions, index=test.index)], axis=1)
    combined.columns = ['actual', 'predictions']
    return error, combined

In [77]:
error, combined = create_predictions(predictors, df_ya, reg)
error

3.1344508952811934

In [78]:
df_ya.tail()

,sum,mean,max,min,target
datetime,,,,,
2024-08-20,24333.3,1013.887500,1015.7,1011.8,1020.8
2024-08-21,24453.0,1018.875000,1020.8,1015.4,1019.0
2024-08-22,24367.8,1015.325000,1019.0,1012.6,1015.6
2024-08-23,24341.8,1014.241667,1015.6,1013.4,1015.6
2024-08-24,24298.2,1012.425000,1015.6,1008.3,1024.1


In [79]:
empty_data = {col: ['' for _ in range(7)] for col in df_ya.columns}
empty_df = pd.DataFrame(empty_data)

df_ya = pd.concat([df_ya, empty_df], ignore_index=False)
df_ya.tail(20)

,sum,mean,max,min,target
2024-08-12 00:00:00,24319.0,1013.291667,1015.5,1011.2,1013.2
2024-08-13 00:00:00,24253.6,1010.566667,1013.2,1007.8,1017.1
2024-08-14 00:00:00,24322.7,1013.445833,1017.1,1011.3,1019.1
2024-08-15 00:00:00,24424.4,1017.683333,1019.1,1016.3,1018.7
2024-08-16 00:00:00,24412.3,1017.179167,1018.7,1014.7,1016.7
2024-08-17 00:00:00,24335.0,1013.958333,1016.7,1011.7,1017.0
2024-08-18 00:00:00,24303.0,1012.625,1017.0,1010.3,1018.3
2024-08-19 00:00:00,24407.6,1016.983333,1018.3,1015.8,1015.7
2024-08-20 00:00:00,24333.3,1013.8875,1015.7,1011.8,1020.8
2024-08-21 00:00:00,24453.0,1018.875,1020.8,1015.4,1019.0


In [73]:
for i in range(5):
    df_ya.iloc[(-7)-i,:]['max'] = df_ya.iloc[(-8)-i,:]['target']
    
df_ya.tail(10)

,sum,mean,max,min,target
2024-08-22 00:00:00,24367.8,1015.325,1019.0,1012.6,1015.6
2024-08-23 00:00:00,24341.8,1014.241667,1015.6,1013.4,1015.6
2024-08-24 00:00:00,24298.2,1012.425,1015.6,1008.3,1024.1
0,,,1024.1,,
1,,,,,
2,,,,,
3,,,,,
4,,,,,
5,,,,,
6,,,,,


In [66]:
df_ya.tail(10)

,sum,mean,max,min,target
2024-08-22 00:00:00,24367.8,1015.325,1019.0,1012.6,1015.6
2024-08-23 00:00:00,24341.8,1014.241667,1015.6,1013.4,1015.6
2024-08-24 00:00:00,24298.2,1012.425,1015.6,1008.3,1024.1
0,,,1024.1,,
1,,,1024.1,,
2,,,,,
3,,,,,
4,,,,,
5,,,,,
6,,,,,


In [65]:
df_ya.iloc[-6,:]['max'] = df_ya.iloc[-8,:]['target']

In [80]:
combined

,actual,predictions
datetime,,
2022-01-01,1025.5,1027.396994
2022-01-02,1022.7,1025.672740
2022-01-03,1009.8,1018.123009
2022-01-04,1018.2,1012.973288
2022-01-05,1023.2,1016.163821
...,...,...
2024-08-20,1020.8,1016.424497
2024-08-21,1019.0,1019.508713
2024-08-22,1015.6,1019.642024
